In [1]:
!pip install transformers
!pip install peft
!pip install accelerate
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install datasets


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip
Looking in indexes: https://pypi.org/simple/

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip show accelerate

Name: accelerate
Version: 0.28.0
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: /usr/local/lib/python3.10/dist-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: peft


In [ ]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel

# Loading the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-8x7B-Instruct-v0.1",
                                             load_in_4bit=True,
                                             torch_dtype=torch.float16,
                                             device_map="auto",
                                             low_cpu_mem_usage=True,
                                             force_download=True, 
                                             resume_download=False
                                             )

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

...

model-00016-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
import json

max_len = 2048
tokenizer.eos_token = "!"

def tokenize(prompt):
    return tokenizer(
        prompt + tokenizer.eos_token,
        truncation=True,
        max_length=max_len
    )
    
def generate_prompt(user_query, sep="\n\n### "):
    p = "<s> [INST]\n" + user_query["instruction"] + user_query["input"] + "[/INST]" + str(user_query["output"]) + "</s>"
    return p

# Load dataset from JSONL file
def load_jsonl_dataset(file_path):
    dataset = []
    with open(file_path, 'r') as file:
        for line in file:
            data = json.loads(line.strip())
            dataset.append(data)
    return dataset

# Load your JSONL dataset
train_data = load_jsonl_dataset("ludwig_train_data_mixtral.jsonl")

# Tokenize function
def tokenize_example(example):
    prompt = generate_prompt(example)
    return tokenize(prompt)

# Tokenize dataset
tokenized_data = [tokenize_example(example) for example in train_data]


In [ ]:
from datasets import load_dataset
train_data = load_dataset('json', data_files='ludwig_train_data_mixtral.jsonl')


In [ ]:
new_model = "RLMixtral"

LORA_R = 8
LORA_ALPHA = 2 * LORA_R
LORA_DROPOUT = 0.1

print("Creating Config...")
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[ "w1", "w2", "w3"],  # Only Training the "expert" layers
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

print("Retrieving Model...")
model = get_peft_model(model, config)

def print_trainable_parameters(m):
    trainable_params = sum(p.numel() for p in m.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in m.parameters())
    print(f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}")

print_trainable_parameters(model)

print("Dataset", train_data)

train_data = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)), remove_columns=["instruction" , "output", "input])

print("Prepare Trainer...")
trainer = Trainer(
    model=model,
    train_dataset=train_data,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        learning_rate=1e-4,
        logging_steps=10,
        optim="adamw_torch",
        save_strategy="epoch",
        output_dir='./results'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False

# Train model
trainer.train()
# Save trained model
trainer.model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# # Push them to the HF Hub
# trainer.model.push_to_hub(new_model, use_temp_dir=False, token="")
# tokenizer.push_to_hub(new_model, use_temp_dir=False, token="")